In [11]:
# . env/bin/activate
import json
import os
import numpy as np
import sklearn
import sklearn.metrics
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# import networkx as nx
import datetime
# filename = "../../mpd/data/mpd.slice.0-999.json"
# filename = "../../mpd.slice.0-999.json"
# data_dir = "../../mpd/data/"
data_dir = "/Users/pani/mpd/data/"

In [12]:
data_files = list(sorted(os.listdir(data_dir)))
# data_files[0]

In [14]:
data_files = data_files[:200]
# len(data_files)

200

In [15]:
all_playlists = []
for file_number, data_file in enumerate(data_files):
#     print(file_number, data_file)
    with open(data_dir + data_file) as file:
        data = json.load(file)
        playlists = data['playlists']
        all_playlists += playlists

In [17]:
playlist_list = all_playlists
len(playlist_list)

200000

In [18]:
import sys
sys.getsizeof(all_playlists)

1764120

In [19]:
def get_tracks(playlist):
    track_list = set([])
    for i in range(playlist['num_tracks']):
        track_list.add(playlist['tracks'][i]['track_uri'])
    return track_list

In [20]:
# get_tracks(playlist_list[0])

In [21]:
# len(playlist_list)

In [ ]:
a b c d e f g

context size 2
training-data:
a b c
b c d a
c a b d e
d b c e f
e c d f g
f d e g
g e f

In [23]:
def get_context_list(tracks_in_playlist, j, context_size=2):
    context_track_list = tracks_in_playlist[max(0, j-context_size):j]
    context_track_list += tracks_in_playlist[j+1:j+context_size+1]
    return context_track_list


context_mapping = {}
for i in range(len(playlist_list)):
    tracks_in_playlist = get_tracks(playlist_list[i])
    tracks_in_playlist = list(tracks_in_playlist)
    for j in range(len(tracks_in_playlist)):
        for context in get_context_list(tracks_in_playlist, j):
            if context in context_mapping.keys():
                context_mapping[context] += 1
            else:
                context_mapping[context] = 1
with open("context-mapping.txt", "w") as text_file:
    for context, count in context_mapping.items():
        text_file.write("%s %d" % (context, count))
        text_file.write("\n")

KeyboardInterrupt: 

In [ ]:
track_mapping = {}
for i in range(len(playlist_list)):
    for item in get_tracks(playlist_list[i]):
        if item in track_mapping.keys():
            track_mapping[item] += 1
        else:
            track_mapping[item] = 1
with open("track-mapping.txt", "w") as text_file:
    for track_uri, count in track_mapping.items():
        text_file.write("%s %d" % (track_uri, count))
        text_file.write("\n")

In [ ]:

with open("training-data.txt", "w") as text_file:
    for i in range(len(playlist_list)):
        tracks_in_playlist = get_tracks(playlist_list[i])
        tracks_in_playlist = list(tracks_in_playlist)
        for j in range(len(tracks_in_playlist)):
            for k in range(len(get_context_list(tracks_in_playlist, j))):
                item = tracks_in_playlist[j] + ' ' + get_context_list(tracks_in_playlist, j)[k]
                text_file.write("%s " % item)
                text_file.write("\n")

In [70]:
# x = [1,2,3,4,5,6]
# context_size = 2
# i = 4
# context = x[max(0, i-context_size):i] + x[i+1:i+context_size+1]
# context

[3, 4, 6]

In [269]:
!cd yoavgo-word2vecf-0d8e19d2f2c6; make
!cd yoavgo-word2vecf-0d8e19d2f2c6; ./word2vecf -train ../training-data.txt -wvocab ../track-mapping.txt -cvocab ../context-mapping.txt -output ../word2vecf-features.txt -size 200 -negative 15 -threads 4

make: Nothing to be done for `all'.
Starting training using file ../training-data.txt
Vocab size: 457016
Word count: 3303932
Vocab size: 457016
Word count: 12915728
thread 3 726509775
thread 1 242169975
thread 2 484339875
thread 0 75
Alpha: 0.000002  Progress: 389.88%  Words/thread/sec: 64.95k  

In [25]:
track_df = pd.read_csv("word2vecf-features.txt", skiprows=1, header=None, sep=' ')
track_df = track_df.set_index(0)
track_df = track_df.drop(columns=[track_df.shape[1]])
track_df.columns

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            191, 192, 193, 194, 195, 196, 197, 198, 199, 200],
           dtype='int64', length=200)

In [26]:
track_df.shape

(457016, 200)

In [28]:
track_df.head()

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
0,,,,,,,,,,,,,,,,,,,,,
spotify:track:0uqPG793dkDDN7sCUJJIVC,-0.026026,-0.083628,0.134833,0.012421,-0.159434,0.083758,0.001227,0.001863,-0.146504,-0.051575,...,-0.039538,0.141645,-0.068446,-0.015492,0.070452,0.078635,0.020971,-0.100852,0.051009,0.062138
spotify:track:7KXjTSCq5nL1LoYtL7XAwS,-0.060636,-0.199347,0.314352,0.025642,-0.353024,0.195261,-0.004815,0.002842,-0.332577,-0.112377,...,-0.096747,0.329734,-0.157763,-0.042936,0.174929,0.186213,0.051125,-0.233378,0.112793,0.141836
spotify:track:1xznGGDReH1oQq0xzbwXa3,-0.054589,-0.198523,0.304365,0.030382,-0.352851,0.190760,-0.004005,-0.000292,-0.327377,-0.113400,...,-0.096744,0.326155,-0.152823,-0.043379,0.171734,0.178939,0.046186,-0.229613,0.109953,0.145087
spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,-0.052947,-0.183444,0.292234,0.033237,-0.349599,0.189332,-0.005676,0.005607,-0.321430,-0.123372,...,-0.097007,0.315117,-0.155333,-0.035356,0.174856,0.180012,0.045895,-0.226134,0.112212,0.136915
spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,-0.055421,-0.196512,0.303226,0.034056,-0.365354,0.192383,0.004406,0.005574,-0.333833,-0.114642,...,-0.089055,0.321614,-0.157704,-0.041348,0.168048,0.183240,0.048365,-0.223629,0.109537,0.140058


In [29]:
# playlist_list[0]
mean_embedding = track_df.mean()
mean_embedding.shape

(200,)

In [30]:
track_array = track_df.values
# track_array is track_array.shape[0] x track_array.shape[1] ...
track_row_names = dict(zip(track_df.index, range(track_array.shape[0])))

count_missing_embeddings = 0

def get_playlist_df(playlist):
    global count_missing_embeddings
    track_uri_list = get_tracks(playlist)
    playlist_dict = {}
    for track_uri in track_uri_list:
        if track_uri not in track_row_names:
            count_missing_embeddings += 1
            track_vec = mean_embedding
        else:
            track_vec = track_array[track_row_names[track_uri], :]
        playlist_dict[track_uri] = track_vec
    return pd.DataFrame(playlist_dict).T

In [ ]:
# def get_playlist_df2(playlist):
#     track_list = get_tracks(playlist)
# #     for set order does not matter. Here is the solution to problem *. & : intersection
# # store the result in track list, so if there is nothing in common, it does not return anything
#     track_list = track_list & set(track_df.index)
#     playlist_df = track_df.loc[list(track_list), :]
# #     problem * sth in track_df.loc[list(track_list), :] is not in the index of track_df(like playlist_list[347]). solution:
    
#     return playlist_df

In [22]:
%timeit get_playlist_df2(playlist_list[0])

88.6 ms ± 16.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit get_playlist_df(playlist_list[0])

1.35 ms ± 73.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
get_playlist_df(playlist_list[0]).shape

(51, 200)

In [163]:
# get_playlist_df(playlist_list[347])

In [31]:
from tqdm import tqdm_notebook as tqdm

In [48]:
from collections import defaultdict
def make_track_playlists(playlist_list):
    track_playlists = defaultdict(set)
    for playlist_index, playlist in enumerate(playlist_list):
        for track_uri in get_tracks(playlist):
            track_playlists[track_uri].add(playlist_index)
    return track_playlists

limited_playlist_list = playlist_list[:10000]
track_playlists = make_track_playlists(limited_playlist_list)
count_missing_embeddings = 0
playlist_df_list = [get_playlist_df(playlist) for index, playlist in enumerate(tqdm(limited_playlist_list))]
print('num missing embedding:', count_missing_embeddings)


('num missing embedding:', 0)


In [33]:
training_playlist = playlist_df_list

In [34]:
len(training_playlist[0])

51

In [35]:
# fit in pandas needs a 2 dim-matrix, with rows being sample and columns being features
# training_playlist is a list of 2-dim-matrix, so if the rows can be on top of eachother
# the 2-dim matrix is build. solution: concat
concat_training_playlist = pd.concat(training_playlist).drop_duplicates()

In [36]:
def make_query_mediod(investigated_playlist, index_to_remove):
    query = investigated_playlist.drop(investigated_playlist.index[index_to_remove])
    query_distance_matrix = sklearn.metrics.pairwise.cosine_distances(query)
    mediod = np.argmin(query_distance_matrix.sum(axis = 0))
    return mediod

In [37]:
def make_mediod_features(mediod, investigated_playlist):
    mediod_features = investigated_playlist.iloc[mediod]
    return mediod_features

In [38]:
from sklearn.neighbors import NearestNeighbors
from tqdm import tnrange
import random

def cal_knn(k, concat_training_playlist):
    nbrs = NearestNeighbors(n_neighbors=k, metric='cosine', n_jobs=1).fit(concat_training_playlist)
    all_true_indices = []
    all_distances = []
    all_indices = []
#     for i in tnrange(len(training_playlist)):
    for i in tnrange(1000):
        if len(training_playlist[i]) < 5:
            continue
        j = random.choice(range(len(training_playlist[i])))
#         for j in range(len(training_playlist[i])):
        track_uri = training_playlist[i].iloc[j].name
        true_index = concat_training_playlist.index.get_loc(track_uri)
        all_true_indices.append(true_index)
        mediod = make_query_mediod(training_playlist[i], j)
        mediod_features = make_mediod_features(mediod, training_playlist[i])
#         distances, indices = nbrs.kneighbors(mediod_features.as_matrix().reshape(1,50))
        distances, indices = nbrs.kneighbors(mediod_features.as_matrix().reshape(1,200))
        all_distances.append(distances)
        all_indices.append(indices)
    return all_true_indices, all_distances, all_indices

In [39]:
all_true_indices, all_distances, all_indices = cal_knn(600, concat_training_playlist)

/Users/pani/work/env/lib/python2.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [96]:
# import pdb
# pdb.pm()

In [40]:
# evaluation metric
def cal_ave_precision(k, all_true_indices, all_indices):
    sum_precision = 0.0
    for i in range(len(all_true_indices)):
        precision = float(all_true_indices[i] in all_indices[i][0, :k])
        sum_precision += precision
    average_precision = sum_precision / len(all_true_indices)
    return average_precision

In [41]:
for k in [10, 400, 500, 600]:
    print(k, 100* cal_ave_precision(k, all_true_indices, all_indices))

(10, 3.9)
(400, 12.2)
(500, 13.8)
(600, 15.0)


In [42]:
# 2.perfect answer(the returned track is the exact dropped track) gain = 2
# 1.a good answer(The found track is in the same playlist as the dropped track) gain = 1
# 0.an incorrect answer(The found track is irrelevant) gain=0

import math
def dcg(gains):
    value = 0
    for index, gain in enumerate(gains):
        if index == 0:
            value += gain
        else:
            value += gain/math.log(1+index)
    return value


In [43]:
def ndcg(gains):
    ideal_gains = list(sorted(gains))     
    ideal_dcg = dcg(ideal_gains)
    if ideal_dcg == 0:
        return 0.0
    real_dcg = dcg(gains)
    return real_dcg/ideal_dcg

In [44]:
def binary_get_gain(playlist_index, true_index, indices):
    gains = []
    for index in indices:
        if index == true_index:
            gains.append(1)
#         elif is_same_artist(playlist_index, true_index, index):
#         elif is_same_playlist(track_playlists, true_index, index):
#             gains.append(1)
        else:
            gains.append(0)
    return gains

In [45]:
def cal_ndcg(track_playlists, all_true_indices, all_indices):
    result = 0
    count = 0
    for true_index, indices in zip(all_true_indices, all_indices):
        playlist_index = count
        gains = binary_get_gain(playlist_index, true_index, indices[0,:])
        result += ndcg(gains)
        count += 1
    return result / count

In [49]:
cal_ndcg(track_playlists, all_true_indices, all_indices)

0.38904311763449806